In [2]:
import numpy as np

In [3]:
def calculate_profit(x1,x2,y,z,n0,p1,p2,p_product):
    #参数说明
    #n:零件1的数量/零件2的数量
    #p1:零件1的次品率
    #p2:零件2的次品率
    #p_product:成品的次品率
    #一些内置的参数
    C1=2#检测零件1的成本
    C2=3#检测零件2的成本
    C_market=56#单件成品的市场售价
    C_assemble=6#装配单件成品的费用
    C_test_product=3#检测单件成品所需的费用
    C_lose=6#调换一件产品引起的损失
    C_disassemble=5#拆解一件产品所需的费用
    #更新次品率
    p1_update=(1-x1)*p1
    p2_update=(1-x2)*p2
    #成品的总次品率
    P=(1-p1_update)*(1-p2_update)*p_product+p1_update+p2_update-p1_update*p2_update
    n=np.floor(min((1-x1*p1)*n0,(1-x2*p2)*n0))
    profit=C_market*(1-P)*n-x1*C1*n0-x2*C2*n0-C_assemble*n-y*C_test_product*n-(1-y)*C_lose*n*P-z*C_disassemble*n*P
    #更新零件数
    if z==0:
        n_update=0
    elif z==1:
        n_update=np.floor(n*P)
    return profit,n_update,p1_update,p2_update

In [16]:
n0=10000
p1=0.1
p2=0.1
p_product=0.1
S=0
same=0
different=0
for p1 in np.linspace(0.95*p1,1.05*p1,5):
    for p2 in np.linspace(0.95*p2,1.05*p2,5):
        for p_product in np.linspace(0.95*p_product,1.05*p_product,5):
            S+=1
            #第一阶段的生产决策
            result=[]
            for x1 in [0,1]:
                for x2 in [0,1]:
                    for y in [0,1]:
                        for z in [0,1]:
                            profit,n_update,p1_update,p2_update=calculate_profit(x1,x2,y,z,n0,p1,p2,p_product)
                            result.append([x1,x2,y,z,profit,n_update,p1_update,p2_update])
            #第二阶段的决策
            result1=np.empty((8, 16), dtype=object)
            for i in range(1,16,2):
                count=0
                for x1 in [0,1]:
                    for x2 in [0,1]:
                        for y in [0,1]:
                            for z in [0,1]:
                                profit,n_update,p1_update,p2_update=calculate_profit(x1,x2,y,z,result[i][5],result[i][6],result[i][7],p_product)
                                result1[int((i-1)/2),count]=[x1,x2,y,z,profit,n_update,p1_update,p2_update]
                                count+=1
            #第三阶段决策
            result2=np.empty((8,8,16), dtype=object)
            for i in range(8):
                for j in range(1,16,2):
                    count=0
                    for x1 in [0,1]:
                        for x2 in [0,1]:
                            for y in [0,1]:
                                for z in [0,1]:
                                    profit,n_update,p1_update,p2_update=calculate_profit(x1,x2,y,z,result1[i,j][5],result1[i,j][6],result1[i,j][7],p_product)
                                    result2[i,int((j-1)/2),count]=[x1,x2,y,z,profit,n_update,p1_update,p2_update]
                                    count+=1
            result2_value=np.zeros((8,8,16))
            for i in range(8):
                for j in range(8):
                    for k in range(16):
                        result2_value[i,j,k]=result2[i,j,k][4]
            result2_value_max=np.max(result2_value,axis=2)
            result1_value=np.zeros((8,16))
            for i in range(8):
                for j in range(16):
                    result1_value[i,j]=result1[i,j][4]
                    result1_value[:,1::2]=result1_value[:,1::2]+result2_value_max
            result1_value_max=np.max(result1_value,axis=1)
            final_result=[0 for i in range(16)]
            for i in range(16):
                final_result[i]=result[i][4]
                final_result[1::2]=final_result[1::2]+result1_value_max
            max_value=max(final_result)
            max_index = final_result.index(max_value)
            print(S)
            print(f"最终的最大利润为{max_value}")
            print("第一阶段的生产决策为:")
            print(result[max_index])
            if max_index%2:
                max_index_1 = np.argmax(result1_value[int((max_index-1)/2)])
                print("第二阶段的生产决策为:")
                print(result1[int((max_index-1)/2),max_index_1])
            if max_index_1%2:
                max_index_2 = np.argmax(result2_value[int((max_index-1)/2),int((max_index_1-1)/2)])
                print("第三阶段的生产决策为:")
                print(result2[int((max_index-1)/2),int((max_index_1-1)/2),max_index_2])
            if (result[max_index][0:4]==[0,0,0,1])&(result1[int((max_index-1)/2),max_index_1][0:4]==[0,0,0,1])&(result2[int((max_index-1)/2),int((max_index_1-1)/2),max_index_2][0:4]==[1,1,0,0]):
                same+=1
            else:
                different+=1
    

1
最终的最大利润为46059701.204604305
第一阶段的生产决策为:
[0, 0, 0, 1, 326615.80874999997, 2587.0, 0.095, 0.095]
第二阶段的生产决策为:
[0, 0, 0, 1, 84495.50972362499, 669.0, 0.095, 0.095]
第三阶段的生产决策为:
[1, 1, 0, 0, 23341.550000000003, 0, 0.0, 0.0]
2
最终的最大利润为46617177.36749006
第一阶段的生产决策为:
[0, 0, 0, 1, 325243.94187499996, 2608.0, 0.095, 0.095]
第二阶段的生产决策为:
[0, 0, 0, 1, 84823.620041, 680.0, 0.095, 0.095]
第三阶段的生产决策为:
[1, 1, 0, 0, 23632.324999999997, 0, 0.0, 0.0]
3
最终的最大利润为47094261.79465007
第一阶段的生产决策为:
[0, 0, 0, 1, 323872.0749999999, 2628.0, 0.095, 0.095]
第二阶段的生产决策为:
[0, 0, 0, 1, 85113.58130999998, 690.0, 0.095, 0.095]
第三阶段的生产决策为:
[1, 1, 0, 0, 23881.199999999997, 0, 0.0, 0.0]
4
最终的最大利润为47640051.43510975
第一阶段的生产决策为:
[0, 0, 0, 1, 322500.20812500006, 2649.0, 0.095, 0.095]
第二阶段的生产决策为:
[0, 0, 0, 1, 85430.30513231251, 701.0, 0.095, 0.095]
第三阶段的生产决策为:
[1, 1, 0, 0, 24165.93, 0, 0.0, 0.0]
5
最终的最大利润为48179367.45544447
第一阶段的生产决策为:
[0, 0, 0, 1, 321128.34125000006, 2669.0, 0.095, 0.095]
第二阶段的生产决策为:
[0, 0, 0, 1, 85709.15427962501, 712.

In [17]:
print(f"在0.95-1.05波动时,生产决策方案和原方案一致的次数为:{same}")
print(f"在0.95-1.05波动时,生产决策方案和原方案不一致的次数为:{different}")
print(f"在0.95-1.05波动时,决策差异率为:{different/125}")

在0.95-1.05波动时,生产决策方案和原方案一致的次数为:125
在0.95-1.05波动时,生产决策方案和原方案不一致的次数为:0
在0.95-1.05波动时,决策差异率为:0.0


In [18]:
n0=10000
p1=0.1
p2=0.1
p_product=0.1
S=0
same=0
different=0
for p1 in np.linspace(0.9*p1,1.1*p1,5):
    for p2 in np.linspace(0.9*p2,1.1*p2,5):
        for p_product in np.linspace(0.9*p_product,1.1*p_product,5):
            S+=1
            #第一阶段的生产决策
            result=[]
            for x1 in [0,1]:
                for x2 in [0,1]:
                    for y in [0,1]:
                        for z in [0,1]:
                            profit,n_update,p1_update,p2_update=calculate_profit(x1,x2,y,z,n0,p1,p2,p_product)
                            result.append([x1,x2,y,z,profit,n_update,p1_update,p2_update])
            #第二阶段的决策
            result1=np.empty((8, 16), dtype=object)
            for i in range(1,16,2):
                count=0
                for x1 in [0,1]:
                    for x2 in [0,1]:
                        for y in [0,1]:
                            for z in [0,1]:
                                profit,n_update,p1_update,p2_update=calculate_profit(x1,x2,y,z,result[i][5],result[i][6],result[i][7],p_product)
                                result1[int((i-1)/2),count]=[x1,x2,y,z,profit,n_update,p1_update,p2_update]
                                count+=1
            #第三阶段决策
            result2=np.empty((8,8,16), dtype=object)
            for i in range(8):
                for j in range(1,16,2):
                    count=0
                    for x1 in [0,1]:
                        for x2 in [0,1]:
                            for y in [0,1]:
                                for z in [0,1]:
                                    profit,n_update,p1_update,p2_update=calculate_profit(x1,x2,y,z,result1[i,j][5],result1[i,j][6],result1[i,j][7],p_product)
                                    result2[i,int((j-1)/2),count]=[x1,x2,y,z,profit,n_update,p1_update,p2_update]
                                    count+=1
            result2_value=np.zeros((8,8,16))
            for i in range(8):
                for j in range(8):
                    for k in range(16):
                        result2_value[i,j,k]=result2[i,j,k][4]
            result2_value_max=np.max(result2_value,axis=2)
            result1_value=np.zeros((8,16))
            for i in range(8):
                for j in range(16):
                    result1_value[i,j]=result1[i,j][4]
                    result1_value[:,1::2]=result1_value[:,1::2]+result2_value_max
            result1_value_max=np.max(result1_value,axis=1)
            final_result=[0 for i in range(16)]
            for i in range(16):
                final_result[i]=result[i][4]
                final_result[1::2]=final_result[1::2]+result1_value_max
            max_value=max(final_result)
            max_index = final_result.index(max_value)
            print(S)
            print(f"最终的最大利润为{max_value}")
            print("第一阶段的生产决策为:")
            print(result[max_index])
            if max_index%2:
                max_index_1 = np.argmax(result1_value[int((max_index-1)/2)])
                print("第二阶段的生产决策为:")
                print(result1[int((max_index-1)/2),max_index_1])
            if max_index_1%2:
                max_index_2 = np.argmax(result2_value[int((max_index-1)/2),int((max_index_1-1)/2)])
                print("第三阶段的生产决策为:")
                print(result2[int((max_index-1)/2),int((max_index_1-1)/2),max_index_2])
            if (result[max_index][0:4]==[0,0,0,1])&(result1[int((max_index-1)/2),max_index_1][0:4]==[0,0,0,1])&(result2[int((max_index-1)/2),int((max_index_1-1)/2),max_index_2][0:4]==[1,1,0,0]):
                same+=1
            else:
                different+=1
    

1
最终的最大利润为42501275.708719976
第一阶段的生产决策为:
[0, 0, 0, 1, 334892.57, 2464.0, 0.09000000000000001, 0.09000000000000001]
第二阶段的生产决策为:
[0, 0, 0, 1, 82517.52924799999, 607.0, 0.09000000000000001, 0.09000000000000001]
第三阶段的生产决策为:
[1, 1, 0, 0, 21484.84, 0, 0.0, 0.0]
2
最终的最大利润为43504721.95451256
第一阶段的生产决策为:
[0, 0, 0, 1, 332118.435, 2505.0, 0.09000000000000001, 0.09000000000000001]
第二阶段的生产决策为:
[0, 0, 0, 1, 83195.66796749999, 627.0, 0.09000000000000001, 0.09000000000000001]
第三阶段的生产决策为:
[1, 1, 0, 0, 22007.699999999997, 0, 0.0, 0.0]
3
最终的最大利润为44560975.19815006
第一阶段的生产决策为:
[0, 0, 0, 1, 329344.30000000005, 2547.0, 0.09000000000000001, 0.09000000000000001]
第二阶段的生产决策为:
[0, 0, 0, 1, 83883.99321, 648.0, 0.09000000000000001, 0.09000000000000001]
第三阶段的生产决策为:
[1, 1, 0, 0, 22558.199999999997, 0, 0.0, 0.0]
4
最终的最大利润为45593948.1455299
第一阶段的生产决策为:
[0, 0, 0, 1, 326570.1649999999, 2588.0, 0.09000000000000001, 0.09000000000000001]
第二阶段的生产决策为:
[0, 0, 0, 1, 84516.358702, 669.0, 0.09000000000000001, 0.09000000000000001]
第

In [20]:
print(f"在0.9-1.1波动时,生产决策方案和原方案一致的次数为:{same}")
print(f"在0.9-1.1波动时,生产决策方案和原方案不一致的次数为:{different}")
print(f"在0.9-1.1波动时,决策差异率为:{different/125}")

在0.9-1.1波动时,生产决策方案和原方案一致的次数为:70
在0.9-1.1波动时,生产决策方案和原方案不一致的次数为:55
在0.9-1.1波动时,决策差异率为:0.44


In [21]:
n0=10000
p1=0.1
p2=0.1
p_product=0.1
S=0
same=0
different=0
for p1 in np.linspace(0.8*p1,1.2*p1,5):
    for p2 in np.linspace(0.8*p2,1.2*p2,5):
        for p_product in np.linspace(0.8*p_product,1.2*p_product,5):
            S+=1
            #第一阶段的生产决策
            result=[]
            for x1 in [0,1]:
                for x2 in [0,1]:
                    for y in [0,1]:
                        for z in [0,1]:
                            profit,n_update,p1_update,p2_update=calculate_profit(x1,x2,y,z,n0,p1,p2,p_product)
                            result.append([x1,x2,y,z,profit,n_update,p1_update,p2_update])
            #第二阶段的决策
            result1=np.empty((8, 16), dtype=object)
            for i in range(1,16,2):
                count=0
                for x1 in [0,1]:
                    for x2 in [0,1]:
                        for y in [0,1]:
                            for z in [0,1]:
                                profit,n_update,p1_update,p2_update=calculate_profit(x1,x2,y,z,result[i][5],result[i][6],result[i][7],p_product)
                                result1[int((i-1)/2),count]=[x1,x2,y,z,profit,n_update,p1_update,p2_update]
                                count+=1
            #第三阶段决策
            result2=np.empty((8,8,16), dtype=object)
            for i in range(8):
                for j in range(1,16,2):
                    count=0
                    for x1 in [0,1]:
                        for x2 in [0,1]:
                            for y in [0,1]:
                                for z in [0,1]:
                                    profit,n_update,p1_update,p2_update=calculate_profit(x1,x2,y,z,result1[i,j][5],result1[i,j][6],result1[i,j][7],p_product)
                                    result2[i,int((j-1)/2),count]=[x1,x2,y,z,profit,n_update,p1_update,p2_update]
                                    count+=1
            result2_value=np.zeros((8,8,16))
            for i in range(8):
                for j in range(8):
                    for k in range(16):
                        result2_value[i,j,k]=result2[i,j,k][4]
            result2_value_max=np.max(result2_value,axis=2)
            result1_value=np.zeros((8,16))
            for i in range(8):
                for j in range(16):
                    result1_value[i,j]=result1[i,j][4]
                    result1_value[:,1::2]=result1_value[:,1::2]+result2_value_max
            result1_value_max=np.max(result1_value,axis=1)
            final_result=[0 for i in range(16)]
            for i in range(16):
                final_result[i]=result[i][4]
                final_result[1::2]=final_result[1::2]+result1_value_max
            max_value=max(final_result)
            max_index = final_result.index(max_value)
            print(S)
            print(f"最终的最大利润为{max_value}")
            print("第一阶段的生产决策为:")
            print(result[max_index])
            if max_index%2:
                max_index_1 = np.argmax(result1_value[int((max_index-1)/2)])
                print("第二阶段的生产决策为:")
                print(result1[int((max_index-1)/2),max_index_1])
            if max_index_1%2:
                max_index_2 = np.argmax(result2_value[int((max_index-1)/2),int((max_index_1-1)/2)])
                print("第三阶段的生产决策为:")
                print(result2[int((max_index-1)/2),int((max_index_1-1)/2),max_index_2])
            if (result[max_index][0:4]==[0,0,0,1])&(result1[int((max_index-1)/2),max_index_1][0:4]==[0,0,0,1])&(result2[int((max_index-1)/2),int((max_index_1-1)/2),max_index_2][0:4]==[1,1,0,0]):
                same+=1
            else:
                different+=1
    

1
最终的最大利润为35386272.48671994
第一阶段的生产决策为:
[0, 0, 0, 1, 351720.9600000001, 2213.0, 0.08000000000000002, 0.08000000000000002]
第二阶段的生产决策为:
[0, 0, 0, 1, 77835.848448, 489.0, 0.08000000000000002, 0.08000000000000002]
第三阶段的生产决策为:
[1, 1, 0, 0, 17777.96, 0, 0.0, 0.0]
2
最终的最大利润为37474819.03063997
第一阶段的生产决策为:
[0, 0, 0, 1, 346050.08, 2297.0, 0.08000000000000002, 0.08000000000000002]
第二阶段的生产决策为:
[0, 0, 0, 1, 79487.703376, 527.0, 0.08000000000000002, 0.08000000000000002]
第三阶段的生产决策为:
[1, 1, 0, 0, 18864.28, 0, 0.0, 0.0]
3
最终的最大利润为39627598.08159998
第一阶段的生产决策为:
[0, 0, 0, 1, 340379.2, 2382.0, 0.08000000000000002, 0.08000000000000002]
第二阶段的生产决策为:
[0, 0, 0, 1, 81078.32544, 567.0, 0.08000000000000002, 0.08000000000000002]
第三阶段的生产决策为:
[1, 1, 0, 0, 19984.8, 0, 0.0, 0.0]
4
最终的最大利润为41769265.11863993
第一阶段的生产决策为:
[0, 0, 0, 1, 334708.32, 2467.0, 0.08000000000000002, 0.08000000000000002]
第二阶段的生产决策为:
[0, 0, 0, 1, 82572.542544, 608.0, 0.08000000000000002, 0.08000000000000002]
第三阶段的生产决策为:
[0, 0, 0, 0, 21100.246016, 0, 0

In [22]:
print(f"在0.8-1.2波动时,生产决策方案和原方案一致的次数为:{same}")
print(f"在0.8-1.2波动时,生产决策方案和原方案不一致的次数为:{different}")
print(f"在0.8-1.2波动时,决策差异率为:{different/125}")

在0.8-1.2波动时,生产决策方案和原方案一致的次数为:15
在0.8-1.2波动时,生产决策方案和原方案不一致的次数为:110
在0.8-1.2波动时,决策差异率为:0.88
